<a href="https://colab.research.google.com/github/SunSlick2/emailSort/blob/main/Email_Sorter_Python_Application_(Version_38_06_Live_Mode_Hourly_Trigger_Refinement).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import win32com.client
import pandas as pd
import datetime
import openpyxl
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
from tkcalendar import Calendar
import threading
import logging
import time
import json
import re
import pythoncom

class EmailSorter:
    """
    A class to sort emails in Outlook based on rules defined in an Excel file
    and configuration from a JSON file.
    Supports live monitoring and bulk processing modes.
    Version 38.06: Implements advanced scheduling for live mode (daily/weekly start, hourly midnight reset),
                   enhanced bulk mode UI (end date selection, 'End Date as Today' checkbox, date validation),
                   substring keyword matching, enhanced HTML body parsing, SMTP cache retry mechanism,
                   and dynamic folder creation/lookup. Fixes live mode initial and hourly trigger logic.
    """

    # Define config file path once here for consistency
    CONFIG_FILE_NAME = 'configv38.05.json'

    def __init__(self, config_path=None):
        """
        Initializes the EmailSorter with configuration, sets up paths,
        loads data from Excel.
        """
        # Use provided config_path or the default class-level constant
        self.config_path = config_path if config_path else self.CONFIG_FILE_NAME

        self.config = None
        self.xls_path = None
        self.log_live_path = None
        self.log_bulk_path = None
        self.log_invalid_path = None

        # Data holders for loaded rules
        self.keyword_subject_to_delete1_keywords = set()
        self.my_cliente_emails = set()
        self.dacs_notmine_emails = set()
        self.my_client_keywords = set()
        self.dacs_notmine_keywords = set()
        self.trade_details_emails = set()
        self.keyword_subject_to_delete_keywords = set() # Original ToDelete list

        self.smtp_cache = {}
        self.new_smtp_entries = {}

        self.live_running = False
        self.invalid_logger = None
        self.live_logger = None
        self.bulk_logger = None

        # Live mode scheduling state
        # Initialized to None to explicitly indicate no check has occurred yet
        self.last_midnight_check_hour = None

        try:
            self._load_config()
            self.setup_paths()
            self.setup_logging()

            self.load_data() # Load all data from Excel

            self.invalid_logger.info("EmailSorter initialized successfully.")
        except Exception as e:
            error_message = f"Initialization error: {e}"
            print(error_message)
            if self.invalid_logger:
                self.invalid_logger.error(f"InitializationError||EmailSorter.__init__|{error_message}")
            messagebox.showerror("Initialization Error", error_message)
            raise

    def _load_config(self):
        """
        Loads configuration from the JSON file.
        Ensures essential paths and sheet mappings are present and correctly formatted.
        """
        try:
            print(f"Attempting to load config from: {os.path.abspath(self.config_path)}")
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)

            print(f"Keys found in sheet_map: {list(self.config['sheet_map'].keys())}")

            required_top_level_keys = ['xls_path', 'log_live_path', 'log_bulk_path', 'log_invalid_path', 'sheet_map']
            for key in required_top_level_keys:
                if key not in self.config:
                    raise ValueError(f"Missing required top-level configuration key: '{key}'")

            # Define expected structure for each rule type in sheet_map
            expected_rule_structure = {
                "KeywordSubject_ToDelete1": {"sheet": str, "columns": list, "match_field": str, "destination_name": str},
                "MyClienteMailAddresses": {"sheet": str, "column": str, "destination_name": str},
                "DACSNotMineEmail": {"sheet": str, "column": str, "destination_name": str},
                "MyClientKeywords": {"sheet": str, "columns": list, "match_field": str, "destination_name": str},
                "DACSNotMineKeyword": {"sheet": str, "columns": list, "match_field": str, "destination_name": str},
                "TradeDetailseMailAddresses": {"sheet": str, "column": str, "destination_name": str},
                "KeywordSubject_ToDelete": {"sheet": str, "columns": list, "match_field": str, "destination_name": str},
                "SMTPResolutionCache": {"sheet": str} # Column key is optional/can be null
            }

            for rule_name, required_keys in expected_rule_structure.items():
                if rule_name not in self.config['sheet_map']:
                    raise ValueError(f"Missing required rule configuration in sheet_map: '{rule_name}'")

                rule_config = self.config['sheet_map'][rule_name]
                for key, expected_type in required_keys.items():
                    if key not in rule_config:
                        # Allow 'column' to be missing if 'columns' is expected, or vice-versa
                        if (key == "column" and "columns" in rule_config) or \
                           (key == "columns" and "column" in rule_config):
                            continue # One of them is present, which is fine
                        raise ValueError(f"Missing required key '{key}' for rule '{rule_name}' in sheet_map.")
                    if not isinstance(rule_config[key], expected_type):
                        # Special handling for column/columns
                        if (key == "columns" and not (isinstance(rule_config[key], list) and len(rule_config[key]) > 0)) or \
                           (key == "column" and not isinstance(rule_config[key], str)):
                            raise ValueError(f"Invalid type for key '{key}' in rule '{rule_name}'. Expected {expected_type.__name__}.")

                # Specific checks for match_field if present
                if "match_field" in rule_config and rule_config["match_field"] not in ['subject_only', 'subject_and_body']:
                    raise ValueError(f"Invalid 'match_field' for '{rule_name}'. Must be 'subject_only' or 'subject_and_body'.")

            print(f"Configuration loaded successfully from {self.config_path}")
        except FileNotFoundError:
            raise FileNotFoundError(f"Configuration file {self.config_path} not found.")
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON in configuration file: {e}")
        except ValueError as e:
            raise e

    def setup_paths(self):
        """Sets up file paths from the loaded configuration."""
        self.xls_path = self.config['xls_path']
        self.log_live_path = self.config['log_live_path']
        self.log_bulk_path = self.config['log_bulk_path']
        self.log_invalid_path = self.config['log_invalid_path']

        os.makedirs(os.path.dirname(self.log_live_path) or '.', exist_ok=True)
        os.makedirs(os.path.dirname(self.log_bulk_path) or '.', exist_ok=True)
        os.makedirs(os.path.dirname(self.log_invalid_path) or '.', exist_ok=True)

        print(f"Paths set: Excel='{self.xls_path}', LiveLog='{self.log_live_path}', BulkLog='{self.log_bulk_path}', InvalidLog='{self.log_invalid_path}'")

    def setup_logging(self):
        """Configures logging for live, bulk, and invalid email entries."""
        self.invalid_logger = self._create_logger('invalid_log', self.log_invalid_path, level=logging.ERROR)
        self.live_logger = self._create_logger('live_log', self.log_live_path, level=logging.INFO)
        self.bulk_logger = self._create_logger('bulk_log', self.log_bulk_path, level=logging.INFO)
        print("Logging setup complete.")

    def _create_logger(self, name, log_path, level=logging.INFO):
        """Helper to create and configure a logger."""
        logger = logging.getLogger(name)
        logger.setLevel(level)

        if logger.handlers:
            for handler in list(logger.handlers):
                logger.removeHandler(handler)

        handler = logging.FileHandler(log_path, mode='a', encoding='utf-8')
        formatter = logging.Formatter('%(asctime)s|%(levelname)s|%(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

    def load_data(self):
        """
        Loads all necessary data (email addresses, keywords, SMTP cache)
        from the configured Excel file based on the sheet_map.
        """
        try:
            self.tables = pd.read_excel(self.xls_path, sheet_name=None, engine='openpyxl')
            print(f"Excel file '{self.xls_path}' loaded successfully.")
        except FileNotFoundError:
            error_msg = f"Excel file not found: {self.xls_path}"
            self.invalid_logger.error(f"FileLoadError||load_data|{error_msg}")
            raise FileNotFoundError(error_msg)
        except Exception as e:
            error_msg = f"Error reading Excel file: {e}"
            self.invalid_logger.error(f"ExcelReadError||load_data|{error_msg}")
            raise ValueError(error_msg)

        # Load data for each rule based on its type in the config
        try:
            self.keyword_subject_to_delete1_keywords = self._load_keywords('KeywordSubject_ToDelete1')
            print(f"Loaded {len(self.keyword_subject_to_delete1_keywords)} 'to delete 1' subject keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|KeywordSubject_ToDelete1|load_data|{e}")
            print(f"Warning: Could not load KeywordSubject_ToDelete1: {e}")

        try:
            self.my_cliente_emails = self._load_email_addresses('MyClienteMailAddresses')
            print(f"Loaded {len(self.my_cliente_emails)} client email addresses.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|MyClienteMailAddresses|load_data|{e}")
            print(f"Warning: Could not load MyClienteMailAddresses: {e}")

        try:
            self.dacs_notmine_emails = self._load_email_addresses('DACSNotMineEmail')
            print(f"Loaded {len(self.dacs_notmine_emails)} non-mine email addresses.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|DACSNotMineEmail|load_data|{e}")
            print(f"Warning: Could not load DACSNotMineEmail: {e}")

        try:
            self.my_client_keywords = self._load_keywords('MyClientKeywords')
            print(f"Loaded {len(self.my_client_keywords)} my client keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|MyClientKeywords|load_data|{e}")
            print(f"Warning: Could not load MyClientKeywords: {e}")

        try:
            self.dacs_notmine_keywords = self._load_keywords('DACSNotMineKeyword')
            print(f"Loaded {len(self.dacs_notmine_keywords)} non-mine keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|DACSNotMineKeyword|load_data|{e}")
            print(f"Warning: Could not load DACSNotMineKeyword: {e}")

        try:
            self.trade_details_emails = self._load_email_addresses('TradeDetailseMailAddresses')
            print(f"Loaded {len(self.trade_details_emails)} trade details email addresses.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|TradeDetailseMailAddresses|load_data|{e}")
            print(f"Warning: Could not load TradeDetailseMailAddresses: {e}")

        try:
            self.keyword_subject_to_delete_keywords = self._load_keywords('KeywordSubject_ToDelete')
            print(f"Loaded {len(self.keyword_subject_to_delete_keywords)} 'to delete' subject keywords.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|KeywordSubject_ToDelete|load_data|{e}")
            print(f"Warning: Could not load KeywordSubject_ToDelete: {e}")

        try:
            self.smtp_cache = self._load_smtp_cache()
            print(f"Loaded {len(self.smtp_cache)} SMTP cache entries.")
        except Exception as e:
            self.invalid_logger.error(f"DataLoadError|SMTPResolutionCache|load_data|{e}")
            print(f"Warning: Could not load SMTPResolutionCache: {e}")

        self.new_smtp_entries = {}

    def _load_email_addresses(self, rule_name):
        """Loads email addresses from a specified Excel sheet and column."""
        sheet_config = self.config['sheet_map'][rule_name]
        sheet_name = sheet_config['sheet']
        column_name = sheet_config['column']

        if sheet_name not in self.tables:
            raise ValueError(f"Sheet '{sheet_name}' not found in Excel file for '{rule_name}'")

        df = self.tables[sheet_name]
        if column_name not in df.columns:
            raise ValueError(f"Column '{column_name}' not found in sheet '{sheet_name}' for '{rule_name}'")

        return set(df[column_name].dropna().astype(str).str.lower())

    def _load_keywords(self, rule_name):
        """
        Loads keywords from a specified Excel sheet and multiple columns.
        Includes stripping leading/trailing whitespace.
        """
        sheet_config = self.config['sheet_map'][rule_name]
        sheet_name = sheet_config['sheet']
        columns = sheet_config['columns'] # Expects a list of columns

        if sheet_name not in self.tables:
            raise ValueError(f"Sheet '{sheet_name}' not found in Excel file for '{rule_name}'")

        df = self.tables[sheet_name]

        missing_columns = [col for col in columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns {missing_columns} not found in sheet '{sheet_name}' for '{rule_name}'")

        keywords = set()
        for column in columns:
            # Added .str.strip() here to remove leading/trailing whitespace from keywords
            column_values = df[column].dropna().astype(str).str.strip().str.lower()
            keywords.update(column_values)

        keywords.discard('')
        return keywords

    def _load_smtp_cache(self):
        """Loads SMTP resolution cache from the 'SMTPResolutionCache' sheet."""
        cache_sheet_name = self.config['sheet_map']['SMTPResolutionCache']['sheet']
        if cache_sheet_name in self.tables:
            cache_df = self.tables[cache_sheet_name]
            if 'EntryName' in cache_df.columns and 'SMTPAddress' in cache_df.columns:
                return dict(zip(
                    cache_df['EntryName'].fillna('').astype(str).str.lower(),
                    cache_df['SMTPAddress'].fillna('').astype(str).str.lower()
                ))
            else:
                self.invalid_logger.warning(
                    f"Missing 'EntryName' or 'SMTPAddress' columns in '{cache_sheet_name}' sheet. "
                    "SMTP cache will not be loaded."
                )
        else:
            self.invalid_logger.warning(
                f"Sheet '{cache_sheet_name}' not found in Excel file. SMTP cache will not be loaded."
            )
        return {}

    def get_smtp_address(self, outlook_namespace, entry):
        """Resolves the SMTP email address for an Outlook recipient or sender entry."""
        if not entry:
            self.invalid_logger.warning("NullEntry||get_smtp_address|Received a None entry.")
            return None

        name = getattr(entry, 'Name', '') or ''
        address = getattr(entry, 'Address', '') or ''
        name_key = (name.lower() if name else address.lower()) or ''

        if not name_key:
            self.invalid_logger.warning(f"EmptyNameKey|Name: '{name}', Address: '{address}'|get_smtp_address|No usable identifier for SMTP lookup.")
            return None

        cached = self.smtp_cache.get(name_key)
        if cached:
            return cached

        smtp = None
        try:
            smtp = entry.PropertyAccessor.GetProperty("http://schemas.microsoft.com/mapi/proptag/0x39FE001E")
            if smtp:
                smtp = smtp.lower()
        except Exception:
            pass

        if not smtp and address:
            smtp = address.lower()

        if smtp:
            self.new_smtp_entries[name_key] = smtp
            self.smtp_cache[name_key] = smtp
            return smtp
        else:
            self.invalid_logger.info(f"NoSMTPResolution|Name: '{name}', Address: '{address}'|get_smtp_address|Could not resolve SMTP address.")
            return None

    def extract_addresses(self, outlook_namespace, mail):
        """Extracts all relevant email addresses (recipients and sender) from a mail item."""
        recipients = set()
        # to_addresses = set() # This is currently not used but kept for potential future use

        # Extract recipients (To, Cc, Bcc)
        try:
            for rec in mail.Recipients:
                smtp = self.get_smtp_address(outlook_namespace, rec)
                if smtp:
                    recipients.add(smtp)
                    # if rec.Type == 1: # olTo - Not used for current logic
                    #     to_addresses.add(smtp)
        except Exception as e:
            self.invalid_logger.error(f"RecipientParseError|{mail.Subject or 'NoSubject'}|extract_addresses|{e}")

        # Extract sender
        sender = None
        try:
            sender = self.get_smtp_address(outlook_namespace, mail.Sender)
            if sender:
                recipients.add(sender)
        except Exception as e:
            self.invalid_logger.error(f"SenderParseError|{mail.Subject or 'NoSubject'}|extract_addresses|{e}")

        if not recipients:
            self.invalid_logger.error(f"NoAddressesFound|Subject: '{mail.Subject or 'NoSubject'}'|extract_addresses|No sender or recipient addresses extracted.")
        return recipients


    def _strip_html_tags(self, html_string):
        """
        Strips HTML tags, converts HTML entities, and normalizes whitespace
        to produce clean plain text for matching.
        """
        if not html_string:
            return ""

        # Decode common HTML entities to their character equivalents
        html_string = html_string.replace('&nbsp;', ' ')
        html_string = html_string.replace('&amp;', '&')
        html_string = html_string.replace('&lt;', '<')
        html_string = html_string.replace('&gt;', '>')
        html_string = html_string.replace('&quot;', '"')
        html_string = html_string.replace('&#x27;', "'") # apostrophe
        html_string = html_string.replace('&#39;', "'") # apostrophe
        html_string = html_string.replace('&#8217;', "'") # right single quotation mark
        html_string = html_string.replace('&#8211;', '-') # en dash
        html_string = html_string.replace('&#8212;', '--') # em dash
        html_string = html_string.replace('&mdash;', '--') # em dash
        html_string = html_string.replace('&ndash;', '-') # en dash

        # Remove script and style tags and their content first
        clean_text = re.sub(r'<script[^>]*>.*?</script>', '', html_string, flags=re.DOTALL | re.IGNORECASE)
        clean_text = re.sub(r'<style[^>]*>.*?</style>', '', clean_text, flags=re.DOTALL | re.IGNORECASE)

        # Replace <br> and <p> tags with newlines to preserve some structure
        clean_text = re.sub(r'<br\s*/?>', '\n', clean_text, flags=re.IGNORECASE)
        clean_text = re.sub(r'</p>', '\n\n', clean_text, flags=re.IGNORECASE) # Paragraph end

        # Remove all other HTML tags
        clean_text = re.sub(r'<[^>]*>', '', clean_text)

        # Normalize whitespace: replace multiple spaces/tabs/newlines with a single space
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()

        return clean_text

    def keyword_match(self, mail, keywords, match_field="subject_and_body"):
        """
        Checks if any of the provided keywords matches a phrase/word in the
        subject or body of the email based on match_field, using regex for substring matching.
        It prioritizes the cleaned HTML body content if available.
        """
        try:
            subject = (mail.Subject or "").lower()

            # Get and clean HTML body first
            body_html_cleaned = ""
            try:
                if hasattr(mail, 'HTMLBody') and mail.HTMLBody:
                    body_html_cleaned = self._strip_html_tags(mail.HTMLBody).lower()
            except Exception as e:
                self.invalid_logger.warning(f"HTMLBodyReadError|{mail.Subject or 'NoSubject'}|keyword_match|Failed to read or strip HTMLBody: {e}")

            # Fallback to plain text body if HTML is empty or failed
            body_plain_text = (mail.Body or "").lower()


            target_content_strings = []
            if match_field == "subject_only":
                target_content_strings.append(subject)
            elif match_field == "subject_and_body":
                target_content_strings.append(subject)
                # Use cleaned HTML body if it has content, otherwise fall back to plain text body
                if body_html_cleaned:
                    target_content_strings.append(body_html_cleaned)
                else:
                    target_content_strings.append(body_plain_text)
            else:
                self.invalid_logger.error(f"InvalidMatchField|{mail.Subject or 'NoSubject'}|keyword_match|Unknown match_field: {match_field}. Defaulting to subject_and_body.")
                target_content_strings.append(subject)
                if body_html_cleaned:
                    target_content_strings.append(body_html_cleaned)
                else:
                    target_content_strings.append(body_plain_text)

            for keyword in keywords:
                # Escape the keyword to treat it as a literal string in regex
                # No word boundaries for substring matching
                pattern = re.escape(keyword)
                regex = re.compile(pattern, re.IGNORECASE)

                for content_string in target_content_strings:
                    if regex.search(content_string):
                        return keyword # Return the first matching keyword
            return None
        except Exception as e:
            self.invalid_logger.error(f"KeywordMatchError|{mail.Subject or 'NoSubject'}|keyword_match|{e}")
            return None

    def log_email(self, logger, outlook_namespace, mail, match_info, dest_folder_name):
        """Logs processed email information to the specified logger."""
        try:
            sent_on = mail.SentOn
            date_str = sent_on.strftime("%Y-%m-%d")
            time_str = sent_on.strftime("%H:%M:%S")

            sender_smtp = self.get_smtp_address(outlook_namespace, mail.Sender) or "Unknown"

            subject = (mail.Subject or "NoSubject").replace('|', ' ').replace('\n', ' ').strip()

            log_entry = f"{date_str}|{time_str}|{sender_smtp}|{subject}|{match_info}|{dest_folder_name}"
            logger.info(log_entry)
        except Exception as e:
            self.invalid_logger.error(
                f"LogFormatError|Subject: '{getattr(mail, 'Subject', 'NoSubject') or 'NoSubject'}'|"
                f"log_email|Failed to format log entry: {e}"
            )

    def _get_or_create_outlook_folder(self, outlook_namespace, folder_path):
        """
        Gets an Outlook folder object by its path, creating it and any necessary
        parent folders if they don't exist.
        Path can be nested, e.g., "Inbox\\SubFolder\\SubSubFolder".
        It handles paths starting with standard folder names or implicitly
        creates subfolders under Inbox if no root is specified.
        """
        path_parts = folder_path.split('\\')
        current_folder = None

        # Determine the initial root folder
        first_part_lower = path_parts[0].lower()

        # Try to get default folders by their common names first
        if first_part_lower == "inbox":
            current_folder = outlook_namespace.GetDefaultFolder(6) # olFolderInbox
            path_parts = path_parts[1:] # Remove "Inbox" from parts to process
        elif first_part_lower == "sent items":
            current_folder = outlook_namespace.GetDefaultFolder(5) # olFolderSentMail
            path_parts = path_parts[1:] # Remove "Sent Items" from parts to process
        else:
            # If the path doesn't start with "Inbox" or "Sent Items", assume it's
            # intended as a subfolder of Inbox.
            current_folder = outlook_namespace.GetDefaultFolder(6) # olFolderInbox
            # path_parts remains as is, e.g., ["DACS-My"] will be created directly under Inbox

        # Iterate through the remaining parts of the path, creating folders as needed
        for sub_folder_name in path_parts:
            # Skip empty parts (e.g., if path was "Inbox\\")
            if not sub_folder_name:
                continue

            try:
                # Attempt to get the subfolder
                current_folder = current_folder.Folders.Item(sub_folder_name)
            except Exception:
                # Folder does not exist, create it
                current_folder = current_folder.Folders.Add(sub_folder_name)
                print(f"Created Outlook folder: '{sub_folder_name}' under '{current_folder.Parent.Name}'")
        return current_folder


    def process_email(self, outlook_namespace, mail, logger, folder_objects_map):
        """
        Processes a single email: finds matching rules and moves the email.
        Rule order updated as per user request (38.06).
        Folder objects are passed via folder_objects_map.
        """
        try:
            recipients = self.extract_addresses(outlook_namespace, mail)

            # Rule 1: Keyword in subject ONLY from KeywordSubject_ToDelete1 (highest priority)
            if self.keyword_match(mail, self.keyword_subject_to_delete1_keywords, match_field="subject_only"):
                dest_folder_name = self.config['sheet_map']['KeywordSubject_ToDelete1']['destination_name']
                mail.Move(folder_objects_map[dest_folder_name])
                self.log_email(logger, outlook_namespace, mail, "Matched by KeywordSubject_ToDelete1", dest_folder_name)
                return True

            # Rule 2: Email address in MyClienteMailAddresses
            if any(addr in self.my_cliente_emails for addr in recipients):
                dest_folder_name = self.config['sheet_map']['MyClienteMailAddresses']['destination_name']
                mail.Move(folder_objects_map[dest_folder_name])
                self.log_email(logger, outlook_namespace, mail, "Matched by MyClienteMailAddresses", dest_folder_name)
                return True

            # Rule 3: Email address in DACSNotMineEmail
            if any(addr in self.dacs_notmine_emails for addr in recipients):
                dest_folder_name = self.config['sheet_map']['DACSNotMineEmail']['destination_name']
                mail.Move(folder_objects_map[dest_folder_name])
                self.log_email(logger, outlook_namespace, mail, "Matched by DACSNotMineEmail", dest_folder_name)
                return True

            # Rule 4: Keyword in subject/body from MyClientKeywords
            if self.keyword_match(mail, self.my_client_keywords, match_field="subject_and_body"):
                dest_folder_name = self.config['sheet_map']['MyClientKeywords']['destination_name']
                mail.Move(folder_objects_map[dest_folder_name])
                self.log_email(logger, outlook_namespace, mail, f"Matched by MyClientKeywords", dest_folder_name)
                return True

            # Rule 5: Keyword in subject/body from DACSNotMineKeyword
            if self.keyword_match(mail, self.dacs_notmine_keywords, match_field="subject_and_body"):
                dest_folder_name = self.config['sheet_map']['DACSNotMineKeyword']['destination_name']
                mail.Move(folder_objects_map[dest_folder_name])
                self.log_email(logger, outlook_namespace, mail, f"Matched by DACSNotMineKeyword", dest_folder_name)
                return True

            # Rule 6: Email address in TradeDetailseMailAddresses
            if any(addr in self.trade_details_emails for addr in recipients):
                dest_folder_name = self.config['sheet_map']['TradeDetailseMailAddresses']['destination_name']
                mail.Move(folder_objects_map[dest_folder_name])
                self.log_email(logger, outlook_namespace, mail, "Matched by TradeDetailseMailAddresses", dest_folder_name)
                return True

            # Rule 7: Keyword in subject ONLY from KeywordSubject_ToDelete (lowest priority)
            if self.keyword_match(mail, self.keyword_subject_to_delete_keywords, match_field="subject_only"):
                dest_folder_name = self.config['sheet_map']['KeywordSubject_ToDelete']['destination_name']
                mail.Move(folder_objects_map[dest_folder_name])
                self.log_email(logger, outlook_namespace, mail, f"Matched by KeywordSubject_ToDelete", dest_folder_name)
                return True

            # If no rules matched, log to original folder name
            self.log_email(logger, outlook_namespace, mail, "No matching rules", mail.Parent.Name)
            return False

        except Exception as e:
            subject = getattr(mail, 'Subject', 'Unknown') or 'NoSubject'
            self.invalid_logger.error(f"EmailProcessingError|Subject: '{subject}'|process_email|{e}")
            print(f"Error processing email '{subject}': {e}")
            return False

    def _get_live_mode_start_filter_time(self):
        """
        Determines the start_time_filter for live mode based on current day of week
        and an hourly midnight check.
        """
        now = datetime.datetime.now()
        today_midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)

        # Calculate base start day for daily/weekly lookback
        base_start_day = today_midnight
        weekday = now.weekday() # Monday is 0, Sunday is 6

        if weekday == 0: # Monday, look back to last Friday
            base_start_day = today_midnight - datetime.timedelta(days=3)
        elif weekday == 5: # Saturday, look back to Friday
            base_start_day = today_midnight - datetime.timedelta(days=1)
        elif weekday == 6: # Sunday, look back to Friday
            base_start_day = today_midnight - datetime.timedelta(days=2)
        else: # Tue, Wed, Thu, Fri
            base_start_day = today_midnight - datetime.timedelta(days=1)

        # Flag to indicate if we should do a midnight check this iteration
        trigger_midnight_check = False

        if self.last_midnight_check_hour is None:
            # Always trigger midnight check on the very first run of live mode
            trigger_midnight_check = True
            print("Live mode: Initial (first run) midnight check triggered.")
        elif now.hour != self.last_midnight_check_hour:
            # If the hour has changed since the last midnight check, trigger one
            trigger_midnight_check = True
            print(f"Live mode: Hourly midnight check triggered for new hour {now.hour:02d}.")

        if trigger_midnight_check:
            start_time_filter = datetime.datetime.combine(base_start_day.date(), datetime.time.min)
            self.last_midnight_check_hour = now.hour # Update the last checked hour to the current hour
            print(f"    Processing from {start_time_filter.strftime('%Y-%m-%d %H:%M:%S')}.")
        else:
            # If not a midnight check, use the standard 5-minute lookback
            start_time_filter = now - datetime.timedelta(minutes=5)
            print(f"Live mode: Standard 5-minute lookback. Processing from {start_time_filter.strftime('%Y-%m-%d %H:%M:%S')}.")

        return start_time_filter


    def process_folder(self, outlook_namespace, folder_to_process, logger, date_filter_time, folder_objects_map):
        """
        Processes all emails within a given Outlook folder.
        Filters emails from the specified date/time onwards.
        Emails are processed in reverse order to handle deletion/moving without affecting iteration.
        This method takes outlook_namespace and target folder *objects* as arguments.
        `date_filter_time` is now a datetime.datetime object.
        """
        processed_count = 0
        folder_name = getattr(folder_to_process, 'Name', 'Unknown')
        print(f"Starting processing for folder: {folder_name}")

        try:
            if not hasattr(folder_to_process, 'Items'):
                self.invalid_logger.error(f"InvalidFolder|{folder_name}|process_folder|Folder object has no 'Items' attribute.")
                return 0

            messages = folder_to_process.Items
            messages.Sort("[ReceivedTime]", False) # Sort by ReceivedTime descending

            # Format the datetime object for Outlook's Restrict method
            # Use %I for 12-hour clock, %M for minute, %p for AM/PM
            # Note: Outlook's Restrict is sometimes timezone-sensitive or behaves unpredictably
            # with specific formats. Using a robust format and manual check.
            date_filter_outlook_str = date_filter_time.strftime('%m/%d/%Y %H:%M %p')
            filter_string = f"[ReceivedTime] >= '{date_filter_outlook_str}'"
            print(f"Applying Outlook filter: {filter_string}")

            try:
                # Use ReceivedTime for filter as it's more reliable than SentOn for filtering when email arrived
                filtered_messages = messages.Restrict(filter_string)
                total_messages_to_process = filtered_messages.Count
                print(f"Found {total_messages_to_process} messages in {folder_name} matching date filter {date_filter_time}.")
            except Exception as restrict_error:
                self.invalid_logger.error(f"OutlookFilterError|{folder_name}|process_folder|Failed to apply filter '{filter_string}': {restrict_error}. Processing all messages and filtering manually.")
                print(f"Warning: Failed to apply Outlook filter: {restrict_error}. Processing all messages and filtering manually.")
                filtered_messages = messages # Fallback to all messages, then filter manually below
                total_messages_to_process = messages.Count # Initial count

            current_message_count = filtered_messages.Count

            for i in range(current_message_count, 0, -1):
                try:
                    mail = filtered_messages.Item(i)

                    # Ensure mail.ReceivedTime is timezone-naive for comparison to avoid issues
                    # with PyTime objects which can be timezone-aware depending on system config.
                    mail_received_time_naive = mail.ReceivedTime.replace(tzinfo=None)

                    # Manual datetime check for robustness if Outlook's Restrict failed or is imprecise
                    if mail_received_time_naive < date_filter_time:
                        continue # Skip emails older than the filter time

                    # Pass thread-local Outlook objects to process_email
                    if self.process_email(outlook_namespace, mail, logger, folder_objects_map):
                        processed_count += 1

                except Exception as msg_error:
                    subject = getattr(mail, 'Subject', 'Unknown') or 'NoSubject'
                    self.invalid_logger.error(f"MessageAccessError|Folder: '{folder_name}', Subject: '{subject}'|process_folder|{msg_error}")
                    print(f"Error accessing or processing message in {folder_name}: {msg_error}")
                    continue

        except Exception as e:
            self.invalid_logger.critical(f"FolderProcessingError|{folder_name}|process_folder|{e}")
            print(f"Critical error processing folder {folder_name}: {e}")
            return 0

        return processed_count

    def run_live(self):
        """
        Runs the email sorter in live mode, continuously monitoring and
        processing emails.
        For the first run, it checks from calculated start day (midnight) or last 5 mins.
        The frequency is 1 minute, with an hourly midnight reset.
        """
        self.live_running = True
        print("Starting live mode...")
        self.live_logger.info("Live mode started.")

        outlook_app = None
        outlook_namespace = None

        # Dictionary to hold all destination folder objects
        live_folder_objects = {}

        try:
            pythoncom.CoInitialize()
            outlook_app = win32com.client.Dispatch("Outlook.Application")
            outlook_namespace = outlook_app.GetNamespace("MAPI")

            # Initialize all required Outlook folder objects dynamically from config
            required_dest_paths = set()
            for rule_name, rule_config in self.config['sheet_map'].items():
                if 'destination_name' in rule_config:
                    required_dest_paths.add(rule_config['destination_name'])

            # Also ensure Inbox and Sent Items are available for processing and as potential destinations
            live_folder_objects["Inbox"] = outlook_namespace.GetDefaultFolder(6)
            live_folder_objects["Sent Items"] = outlook_namespace.GetDefaultFolder(5)

            # Get or create all other specified destination folders
            for folder_path in required_dest_paths:
                # If a folder path is already a direct reference to Inbox or Sent Items, don't re-create.
                # The _get_or_create_outlook_folder handles the implied Inbox root.
                if folder_path.lower() != "inbox" and folder_path.lower() != "sent items":
                    live_folder_objects[folder_path] = self._get_or_create_outlook_folder(outlook_namespace, folder_path)


            print("Outlook initialized for live mode thread.")

            while self.live_running:
                # Determine the time filter based on the new scheduling logic
                # The _get_live_mode_start_filter_time now handles the first-run logic
                start_time_filter = self._get_live_mode_start_filter_time()

                processed_inbox = 0
                processed_sent = 0

                try:
                    processed_inbox = self.process_folder(outlook_namespace, live_folder_objects["Inbox"], self.live_logger, start_time_filter, live_folder_objects)
                    processed_sent = self.process_folder(outlook_namespace, live_folder_objects["Sent Items"], self.live_logger, start_time_filter, live_folder_objects)
                except Exception as e:
                    self.invalid_logger.critical(f"LiveModeFatalError||run_live|A critical error occurred in live mode: {e}")
                    print(f"A critical error occurred in live mode: {e}. Stopping.")
                    self.live_running = False

                total_processed = processed_inbox + processed_sent
                if total_processed > 0:
                    print(f"Processed {total_processed} emails in live mode. Sleeping for 60 seconds...")
                else:
                    print("No new emails to process in live mode. Sleeping for 60 seconds...")

                # Sleep for 1 minute (60 seconds)
                for _ in range(60): # Loop for 60 seconds, checking stop_live flag
                    if not self.live_running:
                        print("Live mode stopped by user request during sleep.")
                        break
                    time.sleep(1)

        except Exception as e:
            self.invalid_logger.critical(f"LiveThreadSetupError||run_live|Failed to set up Outlook in live mode thread: {e}")
            print(f"Failed to set up Outlook in live mode thread: {e}. Live mode aborted.")
            self.live_running = False

        finally:
            self.live_logger.info("Live mode stopped.")
            print("Live mode gracefully stopped.")
            # Clean up COM objects for live mode
            if outlook_app:
                # Release specific folder objects first
                for folder_obj in live_folder_objects.values():
                    try: del folder_obj
                    except Exception as e: self.invalid_logger.warning(f"COMObjectCleanup|run_live|Failed to delete folder object: {e}")

                # Then release namespace and app
                try: del outlook_namespace
                except Exception as e: self.invalid_logger.warning(f"COMObjectCleanup|run_live|Failed to delete namespace object: {e}")
                try: del outlook_app
                except Exception as e: self.invalid_logger.warning(f"COMObjectCleanup|run_live|Failed to delete outlook app object: {e}")
            if 'pythoncom' in globals():
                try:
                    pythoncom.CoUninitialize()
                except Exception as e:
                    self.invalid_logger.error(f"COMUninitializeError|run_live|Failed to uninitialize COM: {e}")
                    print(f"Warning: Failed to uninitialize COM in live mode: {e}")

    def stop_live(self):
        """Signals the live mode to stop its execution loop."""
        self.live_running = False
        print("Stopping live mode...")

    def run_bulk(self, start_date, end_date): # Bulk mode now accepts start_date and end_date
        """Runs the email sorter in bulk mode for a specified date range."""
        print(f"Starting bulk processing for date range: {start_date} to {end_date}...")
        self.bulk_logger.info(f"Bulk mode started for date range: {start_date} to {end_date}.")

        outlook_app = None
        outlook_namespace = None

        # Dictionary to hold all destination folder objects for bulk mode
        bulk_folder_objects = {}

        processed_inbox = 0
        processed_sent = 0

        try:
            pythoncom.CoInitialize()
            outlook_app = win32com.client.Dispatch("Outlook.Application")
            outlook_namespace = outlook_app.GetNamespace("MAPI")

            # Initialize all required Outlook folder objects dynamically from config
            required_dest_paths = set()
            for rule_name, rule_config in self.config['sheet_map'].items():
                if 'destination_name' in rule_config:
                    required_dest_paths.add(rule_config['destination_name'])

            # Also ensure Inbox and Sent Items are available for processing and as potential destinations
            bulk_folder_objects["Inbox"] = outlook_namespace.GetDefaultFolder(6)
            bulk_folder_objects["Sent Items"] = outlook_namespace.GetDefaultFolder(5)

            # Get or create all other specified destination folders
            for folder_path in required_dest_paths:
                if folder_path.lower() != "inbox" and folder_path.lower() != "sent items":
                    bulk_folder_objects[folder_path] = self._get_or_create_outlook_folder(outlook_namespace, folder_path)


            print("Outlook initialized for bulk mode thread.")

            # Convert selected dates (datetime.date) to datetime.datetime at midnight
            bulk_start_time_filter = datetime.datetime.combine(start_date, datetime.time.min)
            # End date filter needs to be up to the end of the day, so add almost one day
            bulk_end_time_filter = datetime.datetime.combine(end_date, datetime.time.max) # Max time for the end date

            # Process Inbox
            processed_inbox = self.process_folder_bulk(outlook_namespace, bulk_folder_objects["Inbox"], self.bulk_logger, bulk_start_time_filter, bulk_end_time_filter, bulk_folder_objects)
            # Process Sent Items
            processed_sent = self.process_folder_bulk(outlook_namespace, bulk_folder_objects["Sent Items"], self.bulk_logger, bulk_start_time_filter, bulk_end_time_filter, bulk_folder_objects)

        except Exception as e:
            self.invalid_logger.critical(f"BulkModeFatalError||run_bulk|A critical error occurred in bulk mode: {e}")
            print(f"A critical error occurred in bulk mode: {e}.")

        finally:
            total_processed = processed_inbox + processed_sent
            print(f"Bulk processing completed. Processed {total_processed} emails for {start_date} to {end_date}.")
            self.bulk_logger.info(f"Bulk mode completed for date range: {start_date} to {end_date}. Processed {total_processed} emails.")

            messagebox.showinfo("Bulk Processing Complete",
                                f"Processed {total_processed} emails for {start_date} to {end_date}\n"
                                f"Inbox: {processed_inbox} emails\n"
                                f"Sent Items: {processed_sent} emails")
            # Clean up COM objects for bulk mode
            if outlook_app:
                for folder_obj in bulk_folder_objects.values():
                    try: del folder_obj
                    except Exception as e: self.invalid_logger.warning(f"COMObjectCleanup|run_bulk|Failed to delete folder object: {e}")

                try: del outlook_namespace
                except Exception as e: self.invalid_logger.warning(f"COMObjectCleanup|run_bulk|Failed to delete namespace object: {e}")
                try: del outlook_app
                except Exception as e: self.invalid_logger.warning(f"COMObjectCleanup|run_bulk|Failed to delete outlook app object: {e}")
            if 'pythoncom' in globals():
                try:
                    pythoncom.CoUninitialize()
                except Exception as e:
                    self.invalid_logger.error(f"COMUninitializeError|run_bulk|Failed to uninitialize COM: {e}")
                    print(f"Warning: Failed to uninitialize COM in bulk mode: {e}")

    # New method for bulk mode folder processing with start and end dates
    def process_folder_bulk(self, outlook_namespace, folder_to_process, logger, start_date_time, end_date_time, folder_objects_map):
        """
        Processes all emails within a given Outlook folder for a specific date range.
        Emails are processed in reverse order to handle deletion/moving without affecting iteration.
        """
        processed_count = 0
        folder_name = getattr(folder_to_process, 'Name', 'Unknown')
        print(f"Starting bulk processing for folder: {folder_name} from {start_date_time} to {end_date_time}")

        try:
            if not hasattr(folder_to_process, 'Items'):
                self.invalid_logger.error(f"InvalidFolder|{folder_name}|process_folder_bulk|Folder object has no 'Items' attribute.")
                return 0

            messages = folder_to_process.Items
            messages.Sort("[ReceivedTime]", False) # Sort by ReceivedTime descending

            # Use both start and end date filters for Outlook's Restrict method
            # Ensure proper string formatting for Outlook
            start_date_outlook_str = start_date_time.strftime('%m/%d/%Y %H:%M %p')
            end_date_outlook_str = end_date_time.strftime('%m/%d/%Y %H:%M %p')

            filter_string = f"[ReceivedTime] >= '{start_date_outlook_str}' AND [ReceivedTime] <= '{end_date_outlook_str}'"
            print(f"Applying Outlook filter: {filter_string}")

            try:
                filtered_messages = messages.Restrict(filter_string)
                total_messages_to_process = filtered_messages.Count
                print(f"Found {total_messages_to_process} messages in {folder_name} matching date range filter.")
            except Exception as restrict_error:
                self.invalid_logger.error(f"OutlookFilterError|{folder_name}|process_folder_bulk|Failed to apply filter '{filter_string}': {restrict_error}. Processing all messages and filtering manually.")
                print(f"Warning: Failed to apply Outlook filter: {restrict_error}. Processing all messages and filtering manually.")
                filtered_messages = messages # Fallback to all messages, then filter manually below
                total_messages_to_process = messages.Count # Initial count

            current_message_count = filtered_messages.Count

            for i in range(current_message_count, 0, -1):
                try:
                    mail = filtered_messages.Item(i)

                    mail_received_time_naive = mail.ReceivedTime.replace(tzinfo=None)

                    # Manual datetime check for robustness if Outlook's Restrict failed or is imprecise
                    if not (start_date_time <= mail_received_time_naive <= end_date_time):
                        continue # Skip emails outside the desired range

                    if self.process_email(outlook_namespace, mail, logger, folder_objects_map):
                        processed_count += 1

                except Exception as msg_error:
                    subject = getattr(mail, 'Subject', 'Unknown') or 'NoSubject'
                    self.invalid_logger.error(f"MessageAccessError|Folder: '{folder_name}', Subject: '{subject}'|process_folder_bulk|{msg_error}")
                    print(f"Error accessing or processing message in {folder_name}: {msg_error}")
                    continue

        except Exception as e:
            self.invalid_logger.critical(f"FolderProcessingError|{folder_name}|process_folder_bulk|{e}")
            print(f"Critical error processing folder {folder_name}: {e}")
            return 0

        return processed_count


    def save_smtp_cache(self):
        """
        Saves newly resolved SMTP entries to the 'SMTPResolutionCache' sheet in the Excel file.
        Implements a retry mechanism with a user prompt if the file is locked.
        """
        if not self.new_smtp_entries:
            print("No new SMTP entries to save.")
            return

        saved_successfully = False
        while not saved_successfully:
            try:
                # Attempt to load the workbook
                wb = openpyxl.load_workbook(self.xls_path, keep_vba=True)

                cache_sheet_name = self.config['sheet_map']['SMTPResolutionCache']['sheet']
                if cache_sheet_name not in wb.sheetnames:
                    ws = wb.create_sheet(cache_sheet_name)
                    ws.append(['EntryName', 'SMTPAddress'])
                    print(f"Created new sheet '{cache_sheet_name}' for SMTP cache.")
                else:
                    ws = wb[cache_sheet_name]
                    # Check if headers exist, if not, add them (handles empty sheet or sheet without headers)
                    if ws.max_row == 0 or ws.cell(row=1, column=1).value not in ['EntryName', 'entryname', 'EntryName']:
                        ws.insert_rows(1)
                        ws.cell(row=1, column=1, value='EntryName')
                        ws.cell(row=1, column=2, value='SMTPAddress')
                        print(f"Added headers to existing sheet '{cache_sheet_name}'.")

                existing_entries = set()
                # Read existing entries starting from the second row (after headers)
                for row in ws.iter_rows(min_row=2, values_only=True):
                    if row and row[0]:
                        existing_entries.add(str(row[0]).lower())

                entries_added = 0
                for entry_name, smtp_address in self.new_smtp_entries.items():
                    if entry_name not in existing_entries:
                        ws.append([entry_name, smtp_address])
                        existing_entries.add(entry_name)
                        entries_added += 1
                    else:
                        print(f"Skipping existing SMTP cache entry: {entry_name}")

                if entries_added > 0:
                    wb.save(self.xls_path)
                    print(f"Saved {entries_added} new SMTP entries to '{cache_sheet_name}' in '{self.xls_path}'.")
                else:
                    print("No *new* unique SMTP entries were added to the cache.")

                wb.close()
                self.new_smtp_entries.clear()
                saved_successfully = True # Set flag to exit loop

            except PermissionError as e:
                error_msg = f"Failed to save SMTP cache: Excel file is currently open. Error: {e}"
                self.invalid_logger.error(f"CacheSaveError||save_smtp_cache|{error_msg}")
                print(error_msg)

                # Prompt user to close the file
                retry_choice = messagebox.askokcancel(
                    "Excel File Locked",
                    f"The Excel file '{os.path.basename(self.xls_path)}' is currently open.\n"
                    "Please close the file to save the SMTP cache.\n\n"
                    "Click OK to retry, or Cancel to skip saving this time."
                )

                if not retry_choice: # User clicked Cancel
                    print("Saving SMTP cache skipped by user.")
                    self.invalid_logger.info("CacheSaveSkipped|save_smtp_cache|User skipped saving SMTP cache due to file lock.")
                    break # Exit the while loop without saving
                else:
                    # User clicked OK, loop will continue for another attempt
                    print("Retrying save operation...")
                    time.sleep(2) # Give a small pause before retrying

            except Exception as e:
                error_msg = f"An unexpected error occurred while saving SMTP cache: {e}"
                self.invalid_logger.critical(f"CacheSaveError||save_smtp_cache|{error_msg}")
                print(error_msg)
                messagebox.showerror("Cache Save Error", error_msg)
                break # Exit the while loop on unexpected error

    def start_gui(self):
        """Starts the main Tkinter GUI for the Email Sorter."""
        root = tk.Tk()
        root.title("Email Sorter v38.06") # Updated title for version
        root.geometry("350x450") # Increased height for new UI elements
        root.resizable(False, False)
        root.attributes('-topmost', True)

        header_label = tk.Label(root, text="Email Sorter", font=("Arial", 16, "bold"), fg="#333333")
        header_label.pack(pady=15)

        info_label = tk.Label(root, text="Choose operation mode:", font=("Arial", 10), fg="#555555")
        info_label.pack(pady=5)

        def pick_bulk():
            """Handles bulk mode selection, prompting for a date range."""
            cal_win = tk.Toplevel(root)
            cal_win.title("Select Date Range for Bulk Processing")
            cal_win.geometry("350x550") # Adjust size for two calendars and checkbox
            cal_win.resizable(False, False)
            cal_win.attributes('-topmost', True)
            cal_win.grab_set()

            # Start Date selection
            start_date_label = tk.Label(cal_win, text="Select Start Date:", font=("Arial", 10))
            start_date_label.pack(pady=(10, 0))
            cal_start = Calendar(cal_win, selectmode='day', date_pattern='yyyy-mm-dd',
                                background="blue", foreground="white",
                                headersbackground="blue", headersforeground="white",
                                selectbackground="green", selectforeground="white",
                                normalbackground="lightgray", weekendbackground="darkgray")
            cal_start.pack(pady=(0, 10))

            # End Date selection
            end_date_label = tk.Label(cal_win, text="Select End Date:", font=("Arial", 10))
            end_date_label.pack(pady=(10, 0))

            # Checkbox for "End Date as Today"
            end_date_today_var = tk.BooleanVar(value=True) # Defaults to True (checked)
            end_date_checkbox = tk.Checkbutton(cal_win, text="End Date as Today", variable=end_date_today_var,
                                               font=("Arial", 9))
            end_date_checkbox.pack(anchor=tk.W, padx=10) # Align left

            cal_end = Calendar(cal_win, selectmode='day', date_pattern='yyyy-mm-dd',
                              background="blue", foreground="white",
                              headersbackground="blue", headersforeground="white",
                              selectbackground="green", selectforeground="white",
                              normalbackground="lightgray", weekendbackground="darkgray")
            cal_end.pack(pady=(0, 10))

            # Function to toggle end date calendar state based on checkbox
            def toggle_end_date_calendar():
                if end_date_today_var.get():
                    cal_end.config(state='disabled')
                else:
                    cal_end.config(state='normal')
            end_date_today_var.trace_add("write", lambda *args: toggle_end_date_calendar())
            toggle_end_date_calendar() # Call once to set initial state


            def validate_dates_and_process():
                selected_start_date = cal_start.selection_get() # datetime.date object

                if end_date_today_var.get():
                    selected_end_date = datetime.date.today()
                else:
                    selected_end_date = cal_end.selection_get() # datetime.date object

                today = datetime.date.today()

                if selected_start_date > today:
                    messagebox.showerror("Invalid Date", "Start Date cannot be in the future.")
                    return
                if selected_end_date > today:
                    messagebox.showerror("Invalid Date", "End Date cannot be in the future.")
                    return
                if selected_start_date > selected_end_date:
                    messagebox.showerror("Invalid Date Range", "Start Date cannot be after End Date.")
                    return

                cal_win.destroy()
                root.destroy()
                # Pass both start and end dates to run_bulk
                threading.Thread(target=lambda: self.run_bulk(selected_start_date, selected_end_date), daemon=True).start()

            button_frame = tk.Frame(cal_win)
            button_frame.pack(pady=10)

            tk.Button(button_frame, text="Process", command=validate_dates_and_process,
                      bg="#28a745", fg="white", width=12, height=1,
                      font=("Arial", 10, "bold"), relief=tk.RAISED).pack(side=tk.LEFT, padx=10)
            tk.Button(button_frame, text="Cancel", command=cal_win.destroy,
                      bg="#dc3545", fg="white", width=12, height=1,
                      font=("Arial", 10, "bold"), relief=tk.RAISED).pack(side=tk.LEFT, padx=10)

        def pick_live():
            """Handles live mode selection, starting continuous monitoring."""
            root.destroy()

            live_win = tk.Tk()
            live_win.title("Live Mode - Email Sorter")
            live_win.geometry("320x160")
            live_win.resizable(False, False)
            live_win.attributes('-topmost', True)

            status_label = tk.Label(live_win, text="Live monitoring active...",
                                     font=("Arial", 12, "bold"), fg="green")
            status_label.pack(pady=20)

            info_label = tk.Label(live_win, text="Emails are being processed automatically in the background.",
                                   font=("Arial", 9), fg="#666666")
            info_label.pack(pady=5)

            def stop_and_close():
                self.stop_live()
                # Calling save_smtp_cache, which now handles the prompt
                self.save_smtp_cache()
                live_win.destroy()

            tk.Button(live_win, text="Stop Live Mode", command=stop_and_close,
                      bg="#dc3545", fg="white", width=18, height=1,
                      font=("Arial", 10, "bold"), relief=tk.RAISED).pack(pady=20)

            threading.Thread(target=self.run_live, daemon=True).start()

            live_win.protocol("WM_DELETE_WINDOW", stop_and_close)
            live_win.mainloop()

        button_frame = tk.Frame(root)
        button_frame.pack(pady=20)

        tk.Button(button_frame, text="Run Live Mode", command=pick_live,
                  bg="#007bff", fg="white", width=18, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(pady=8)
        tk.Button(button_frame, text="Run Bulk Mode", command=pick_bulk,
                  bg="#ffc107", fg="white", width=18, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(pady=8)

        footer_label = tk.Label(root, text="Live: Monitors emails based on smart schedule | Bulk: Process selected date range",
                                 font=("Arial", 8), fg="gray")
        footer_label.pack(side=tk.BOTTOM, pady=10)

        def on_closing():
            # This prompt also uses the robust save_smtp_cache
            if messagebox.askyesno("Exit", "Do you want to save the SMTP cache before exiting? (Recommended)"):
                self.save_smtp_cache()
            root.destroy()

        root.protocol("WM_DELETE_WINDOW", on_closing)
        root.mainloop()

def main():
    """Main function to run the Email Sorter application."""
    sorter = None
    try:
        # Use the class-level constant for config_path
        sorter = EmailSorter()
        sorter.start_gui()
    except Exception as e:
        print(f"Error starting Email Sorter application: {e}")
        if sorter and sorter.invalid_logger:
            sorter.invalid_logger.critical(f"AppStartupError||main|{e}")
    finally:
        # Final attempt to save cache if main GUI closes in an unexpected way,
        # but the prompt will handle it for user-initiated closes.
        if sorter:
            pass # No longer call save_smtp_cache directly here, it's handled by GUI callbacks


if __name__ == "__main__":
    main()